In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [58]:
#Other necessary imports
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import keras
from keras import models
from keras import layers
from keras.preprocessing.image import ImageDataGenerator

# Initial EDA

In [3]:
#First 5 lines of the output from the for loop above
'''
/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv
/kaggle/input/siim-isic-melanoma-classification/train.csv
/kaggle/input/siim-isic-melanoma-classification/test.csv
/kaggle/input/siim-isic-melanoma-classification/jpeg/test/ISIC_2417927.jpg
/kaggle/input/siim-isic-melanoma-classification/jpeg/test/ISIC_0546632.jpg
'''

In [4]:
root_path = '../input/siim-isic-melanoma-classification/'
print(list(os.listdir(root_path)))

In [5]:
df = pd.read_csv(root_path + 'train.csv')
df.head()

In [6]:
df.info()

In [7]:
df.isna().sum()

In [8]:
print(f"Does Image Name have any duplicates?: {df['image_name'].nunique() != len(df['image_name'])}")

In [9]:
print(f"Does Patient Id have any duplicates?: {df['patient_id'].nunique() != len(df['patient_id'])}")
df['patient_id'].nunique()

In [10]:
df['sex'].value_counts()

In [11]:
df['sex'].value_counts(normalize=True)

In [12]:
df['diagnosis'].value_counts()

In [13]:
df[df['diagnosis']=='melanoma']['target'].value_counts()

In [14]:
df[df['diagnosis']!='melanoma']['target'].value_counts()

In [15]:
#pd.DataFrame(df.groupby(['sex','target'])['benign_malignant'].count())

Create a column for each path, jpeg and dicom <br>
Source: https://www.kaggle.com/andradaolteanu/siim-melanoma-competition-eda-augmentations#4.-Preprocess-.csv-files-%F0%9F%93%90

In [16]:
df['path_dicom'] = root_path + 'train/' + df['image_name'] + '.dcm'
df['path_jpeg'] = root_path + 'jpeg/train/' + df['image_name'] + '.jpg'
print(df['image_name'][0])
print(df['path_jpeg'][0])

Create a dataframe of image sizes. <br> Source: https://www.kaggle.com/andradaolteanu/siim-melanoma-competition-eda-augmentations#5.-The-Images-%F0%9F%93%B8

In [17]:
# shapes_train = []

# for k, path in enumerate(df['path_jpeg']):
#     image = Image.open(path)
#     shapes_train.append(image.size)
        
# shapes_train = pd.DataFrame(data = shapes_train, columns = ['H', 'W'], dtype='object')
# shapes_train['Size'] = '[' + shapes_train['H'].astype(str) + ', ' + shapes_train['W'].astype(str) + ']'

In [18]:
# max_image_height = shapes_train['H'].max()
# max_image_width = shapes_train['W'].max()
# print(max_image_height)
# print(max_image_width)

In [19]:
'''
max_image_height = 6000
max_image_width = 6000
'''

In [20]:
# shapes_train['H'].value_counts().head(10)

In [21]:
'''
OUTPUT FOR IMAGE HEIGHT
6000    14704
1872     7534
640      4147
5184     3418
3264     1483
4288      729
2592      674
3888      140
4032       84
2317       29
Name: H, dtype: int64
'''

In [22]:
# shapes_train['W'].value_counts().head(10)

In [23]:
'''
OUTPUT FOR IMAGE WIDTH
4000    14703
1053     7534
480      4147
3456     3418
2448     1483
2848      729
1936      678
2592      144
3024       84
2317       29
Name: W, dtype: int64
'''

In [24]:
# df['image_size'] = shapes_train['Size']
# print(df[df['target']==0]['image_size'].value_counts().head())
# print(df[df['target']==1]['image_size'].value_counts().head())

In [25]:
#OUTPUT FROM ABOVE CELL
'''
[6000, 4000]    14679
[1872, 1053]     7404
[640, 480]       4056
[5184, 3456]     3346
[3264, 2448]     1391
Name: image_size, dtype: int64
[1872, 1053]    130
[3264, 2448]     92
[640, 480]       91
[5184, 3456]     72
[2592, 1936]     71
Name: image_size, dtype: int64
'''

In [26]:
df['target'].value_counts()

In [27]:
df['target'].value_counts(normalize=True)

The target is heavily unbalanced

Read in the 2019 data <br> Source: https://www.kaggle.com/cdeotte/jpeg-isic2019-1024x1024 <BR> OR: https://www.kaggle.com/cdeotte/jpeg-isic2019-512x512

In [28]:
df_2019 = pd.read_csv("../input/jpeg-isic2019-512x512/train.csv")
df_2019['path_jpeg'] = "../input/jpeg-isic2019-512x512/train/" + df_2019['image_name'] + ".jpg"
print(df_2019['path_jpeg'][0])

In [29]:
df_2019.info()

In [30]:
print(df_2019['target'].value_counts())
print(df_2019['benign_malignant'].value_counts())

In [31]:
df_2019[df_2019['diagnosis']=='MEL']['benign_malignant'].value_counts()

In [32]:
df_2019_malignant = df_2019[df_2019['benign_malignant']=='malignant']
df_2019_malignant

Read in more jpg images that weren't from 2020,2019,2018, or 2017 <br> Source: https://www.kaggle.com/cdeotte/malignant-v2-1024x1024

In [33]:
df_more = pd.read_csv("../input/malignant-v2-1024x1024/train_malig_2.csv")
df_more['path_jpeg'] = "../input/malignant-v2-1024x1024/jpeg1024/" + df_more['image_name'] + ".jpg"
print(df_more['path_jpeg'][0])

In [34]:
df_more.info()

In [35]:
print(df_more['target'].value_counts())
print(df_more['benign_malignant'].value_counts())

In [36]:
df_combined = pd.concat([df, df_2019_malignant, df_more])
df_combined.info()

In [37]:
df_combined['benign_malignant'].value_counts(normalize=True)

# Train Test Split

In [38]:
X = df_combined[['path_jpeg']]
y = df_combined['benign_malignant']
X_use, X_test, y_use, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_use, y_use, test_size=0.25, random_state=42, stratify=y_use)

In [39]:
train_df = pd.concat([X_train, y_train], axis=1)
train_df

In [40]:
val_df = pd.concat([X_val, y_val], axis=1)
val_df

In [41]:
test_df = pd.concat([X_test, y_test], axis=1)
test_df

In [42]:
train_df['benign_malignant'].value_counts(normalize=True)

In [43]:
val_df['benign_malignant'].value_counts(normalize=True)

# Modeling

In [44]:
#Divide datasets by 10 for now
train_generator = ImageDataGenerator(rescale = 1./255).flow_from_dataframe(train_df, 
                                                                           x_col='path_jpeg', 
                                                                           y_col='benign_malignant',
                                                                           seed=42,
                                                                           target_size=(256, 256), 
                                                                           batch_size = int(len(train_df)/10),
                                                                           class_mode='categorical') 
val_generator = ImageDataGenerator(rescale = 1./255).flow_from_dataframe(val_df, 
                                                                         x_col='path_jpeg', 
                                                                         y_col='benign_malignant',
                                                                         seed=42,
                                                                         target_size=(256, 256), 
                                                                         batch_size = int(len(val_df)/10),
                                                                         class_mode='categorical') 
test_generator = ImageDataGenerator(rescale = 1./255).flow_from_dataframe(test_df, 
                                                                          x_col='path_jpeg',
                                                                          y_col='benign_malignant',
                                                                          seed=42,
                                                                          target_size=(256, 256), 
                                                                          batch_size = int(len(test_df)/10),
                                                                          class_mode='categorical')  

In [45]:
# create the data sets
train_images, train_labels = next(train_generator)
val_images, val_labels = next(val_generator)
test_images, test_labels = next(test_generator)

In [46]:
#checking classes
print("Train data set classes",train_generator.class_indices)
print("Validation data set classes",val_generator.class_indices)
print("Test data set classes",test_generator.class_indices)

In [47]:
# Explore your dataset again
print ("Number of training samples: " + str(train_images.shape[0]))
print ("Number of validation samples: " + str(val_images.shape[0]))
print ("Number of testing samples: " + str(test_images.shape[0]))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))

In [48]:
#Reshape the images
train_img = train_images.reshape(train_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)

print(train_img.shape)
print(val_img.shape)
print(test_img.shape)

In [49]:
train_y = np.reshape(train_labels[:,0], (2580,1))
val_y = np.reshape(val_labels[:,0], (860,1))
test_y = np.reshape(test_labels[:,0], (382,1))

In [62]:
# Build a baseline fully connected model
np.random.seed(42)
baseline_model = models.Sequential()
baseline_model.add(layers.Dense(20, activation='relu', input_shape=(196608,))) # 2 hidden layers
baseline_model.add(layers.Dense(7, activation='relu'))
baseline_model.add(layers.Dense(5, activation='relu'))
baseline_model.add(layers.Dense(1, activation='sigmoid'))

In [67]:
baseline_model.compile(optimizer='sgd',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'Recall', 'AUC'])

results = baseline_model.fit(train_img,
                            train_y,
                            epochs=50,
                            batch_size=32,
                            validation_data=(val_img, val_y))

In [52]:
''' Output with images resized to 256x256
81/81 [==============================] - 1s 18ms/step - loss: 0.1611 - accuracy: 0.9442 - 
val_loss: 0.2397 - val_accuracy: 0.9151
'''

In [68]:
#VISUALIZE RESULT
train_loss = results.history['loss']
train_acc = results.history['accuracy']
train_recall = results.history['recall']
train_aucroc = results.history['auc']
val_loss = results.history['val_loss']
val_acc = results.history['val_accuracy']
val_recall = results.history['val_recall']
val_aucroc = results.history['val_auc']


fig, [(ax1, ax2), (ax3, ax4)] = plt.subplots(2, 2, figsize=(10, 5))
sns.lineplot(x=results.epoch, y=train_loss, ax=ax1, label='train_loss')
sns.lineplot(x=results.epoch, y=train_acc, ax=ax2, label='train_accuracy')
sns.lineplot(x=results.epoch, y=train_recall, ax=ax3, label='train_recall')
sns.lineplot(x=results.epoch, y=train_aucroc, ax=ax4, label='train_aucroc')

sns.lineplot(x=results.epoch, y=val_loss, ax=ax1, label='val_loss')
sns.lineplot(x=results.epoch, y=val_acc, ax=ax2, label='val_accuracy')
sns.lineplot(x=results.epoch, y=val_recall, ax=ax3, label='val_recall')
sns.lineplot(x=results.epoch, y=val_aucroc, ax=ax4, label='val_aucroc');

In [70]:
baseline_model_train = baseline_model.evaluate(train_img, train_y)
baseline_model_val = baseline_model.evaluate(val_img, val_y)
baseline_model_results = {'Training': [baseline_model_train[0], baseline_model_train[1], baseline_model_train[2], baseline_model_train[3]], 
                          'Validation': [baseline_model_val[0], baseline_model_val[1], baseline_model_val[2], baseline_model_val[3]]}
baseline_model_results_df = pd.DataFrame.from_dict(data = baseline_model_results,
                                                   orient = 'index', 
                                                   columns = ['Loss', 'Accuracy', 'Recall', 'AUC-ROC'])
baseline_model_results_df.to_csv('baseline_results.csv')